In [68]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [100]:
play_df = pd.read_csv(f'{DATA_PATH}playstore_df.csv')
yogi_df = pd.read_csv(f'{DATA_PATH}yogiyo_reviews_jsi.csv')

play_df.shape, yogi_df.shape

((103650, 3), (18686, 6))

In [101]:
play_df.head()

,content,score,replyContent
0,쿠폰ㅋㅋㅋㅋㅋㅋ 추가) 다른 분이 언급했듯 배민1 쿠폰과 일반 쿠폰 혼재에 따른 편...,2,"o Dan님,\n\n안녕하세요. 배달의민족입니다.\n혹시 이용에 불편함이나, 만족스..."
1,이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것,2,"문상화님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족 이용 후 좋다고 말씀해주..."
2,이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요,1,"풍기도리님,\n\n안녕하세요. 배달의민족입니다.\n할인 혜택이 부족하여 아쉬움을 드..."
3,"아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배...",1,"롤린이배린이 (롤배그올리는채널)님,\n\n안녕하세요. 배달의민족입니다.\n혜택 부족..."
4,좋아요!!,5,"꼬냑소년님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족 서비스 이용에 만족하신..."


In [102]:
# 결측치 확인

play_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103650 entries, 0 to 103649
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   content       103646 non-null  object
 1   score         103650 non-null  int64 
 2   replyContent  103650 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.4+ MB


In [103]:
# 최빈값

play_mode = play_df['content'].mode()[0]
play_mode

'좋아요'

In [104]:
play_df.isnull().sum()

content         4
score           0
replyContent    0
dtype: int64

In [105]:
play_df['content'] = play_df['content'].fillna(play_mode)

In [106]:
play_df.isnull().sum()

content         0
score           0
replyContent    0
dtype: int64

In [107]:
play_df['content'] = play_df.content.str.replace('[^a-zA-Z가-힣0-9 .,!?\'\"]', '', regex=True)
play_df.content

0         쿠폰 추가 다른 분이 언급했듯 배민1 쿠폰과 일반 쿠폰 혼재에 따른 편의성 및 사용...
1               이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것
2                               이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요
3         아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배...
4                                                     좋아요!!
                                ...                        
103645                             통신이 아예 안 되네요 4g는 빵빵 터지는데
103646    배달의 민족 초반때부터 사용했었는데 예전이 오히려 나은거 같아요 위치설정도 왔다리갔...
103647                             가끔 와파떠잇는데 와파잡아야댄다고안들어가저요
103648                             어쩌라는거야.. 서울에서 안산까지 배달해줌?
103649                                         진짜맛없음돈받고광고하나
Name: content, Length: 103650, dtype: object

In [108]:
# 졍규표현식
play_df['replyContent'] = play_df.replyContent.str.replace('[^a-zA-Z가-힣0-9 .,!?\'\"]', '', regex=True)

# 학습에 무의미한 문장 제거
play_df['replyContent'] = play_df.replyContent.str.replace('배달의민족입니다.',"", regex=True)
play_df['replyContent'] = play_df.replyContent.str.replace('배달의민족 입니다.',"", regex=True)

# 개인ID -> 고객님 변경
play_df['replyContent'] = play_df.replyContent.str.replace("\w+님", "고객님", regex=True)
play_df['replyContent'] = play_df.replyContent.str.replace("\w+ 님", "고객님", regex=True)

play_df.replyContent

0         o 고객님,안녕하세요. 혹시 이용에 불편함이나, 만족스럽지 못한 부분을 느끼셨는지요...
1         고객님,안녕하세요. 배달의민족 이용 후 좋다고 말씀해주시니 고객님의 행복함이 가슴 ...
2         고객님,안녕하세요. 할인 혜택이 부족하여 아쉬움을 드린 것 같아 마음이 무겁습니다....
3         롤린이배린이 고객님,안녕하세요. 혜택 부족으로 아쉬움을 드려 죄송합니다.배달의민족은...
4         고객님,안녕하세요. 배달의민족 서비스 이용에 만족하신 것 같아서 정말 기쁩니다앞으로...
                                ...                        
103645    안녕하세요.  네트워크 문제로 불편을 드려 죄송합니다.배달의민족은 네트워크 문제에 ...
103646    오랜 시간 보내주신 사랑을 먹고 무럭무럭 성장하고 있는  업소정보에 실망하셨다니 고...
103647    와아파이 잡아야서 얼른 주문해야 되는데 계속 물어봐서 짜증나셨죠. 네트워크 오류가 ...
103648    안녕하세요 전국구 배달어플  안산에서 서울의 업소정보를 보게 되셨군요. 최근 위치를...
103649    안녕하세요.  주문하신 업소의 음식이 마음에 안드셨다니 안타까운 마음입니다. 배달의...
Name: replyContent, Length: 103650, dtype: object

In [109]:
yogi_df.head()

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,5,5,5,5.0,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,5,5,5,5.0,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,5,5,5.0,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [110]:
for i in range(len(yogi_df[:100])):
    print(i)
    print(yogi_df['고객리뷰'][i])
    print('------------------------------------------------')
    print(yogi_df['사장댓글'][i])

0
사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱 맞춰서 보내주셨어요~~~~
------------------------------------------------
이렇게 저희 매장 찾아주셔서 감사드립니다!!

맛 뿐만 아니라 양도, 서비스도 모두 최상의 품질을 드리려고 노력하고 있습니다🙏 모든 직원들이 힘내서 고객님의 마음을 잡기 위해 계속 노력하겠습니다ㅎㅎ

항상 건강 챙기시고, 오늘도 즐거운 하루 보내시길 바라겠습니다❤                                                
1
리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!
------------------------------------------------
좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇

고객님께서 맛있다고 말씀 주시니 피로가 싹~ 사라지네요! 주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다시 시작할 수 있는 거 같습니다^^ 정말 감사드려요ㅎㅎ

오늘도 화이팅 있게 마무리하시길 바라요💪
2
잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.
------------------------------------------------
소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 

저희를 찾아 주신 만큼 100% 만족감을 드리고 싶었는데 그러지 못한 것 같아 너무 죄송합니다ㅜㅜ 
말씀 주신 부분에 대해서는 주의하여 다음 주문시에는 실수 없도록 꼼꼼하게 확인하겠습니다..!! 

저희 매장을 찾아 주셔서 다시 한번 감사의 말씀드리며, 언제나 행복한 순간들만 가득하시길 바라겠습니다!
3
굿 말이 필요 없는 메뉴 였습니다
------------------------------------------------
수 많은 매장들을 헤치고 와주셔서 감사합니다!!

저희 매장의 메뉴를 제대로 즐기신 거 같아 너무 영광이에요😋 모두 마음에 드셨을까요??전부 정성껏 준비했

In [111]:
# 결측치 확인

yogi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18686 entries, 0 to 18685
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   고객리뷰    18686 non-null  object 
 1   별점      18686 non-null  int64  
 2   맛별점     18686 non-null  int64  
 3   양별점     18686 non-null  int64  
 4   배달별점    18686 non-null  float64
 5   사장댓글    18686 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 876.0+ KB


In [112]:
yogi_df['고객리뷰'] = yogi_df.고객리뷰.str.replace('[^a-zA-Z가-힣0-9 .,!?\'\"]', '', regex=True)
yogi_df.고객리뷰

0           사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요
1                        리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!
2                      잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.
3                                      굿 말이 필요 없는 메뉴 였습니다
4                                             맛있어요 양도많구좋음
                               ...                       
18681                                          맛있게 잘 먹었어요
18682           요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다
18683                                배달 빨라요 맛있게 잘 먹었습니당 !
18684                                              잘 먹었어요
18685    식후감 자주 시키는데 항상 요청사항도 잘들어주시고 맛있고 빨라요 앞으로도 자주 시킬게요
Name: 고객리뷰, Length: 18686, dtype: object

In [113]:
yogi_df['사장댓글'] = yogi_df.사장댓글.str.replace('[^a-zA-Z가-힣0-9 .,!?\'\"]', '', regex=True)
yogi_df.사장댓글

0        이렇게 저희 매장 찾아주셔서 감사드립니다!!맛 뿐만 아니라 양도, 서비스도 모두 최...
1        좋은 리뷰 덕분에 기분이 행복해지는 것 같아요고객님께서 맛있다고 말씀 주시니 피로가...
2        소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3        수 많은 매장들을 헤치고 와주셔서 감사합니다!!저희 매장의 메뉴를 제대로 즐기신 거...
4        수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
                               ...                        
18681                                감사합니다!!! 다음에도 이용해주세용 
18682                                죄송합니다 !! 다음에도 애용해주세용 
18683                                감사합니다!!! 다음에도 이용해주세용 
18684                                감사합니다!!! 다음에도 이용해주세용 
18685                                감사합니다!!! 다음에도 이용해주세용 
Name: 사장댓글, Length: 18686, dtype: object

In [114]:
tmp = {
    'content': yogi_df['고객리뷰'],
    'score': yogi_df['별점'],
    'replyContent': yogi_df['사장댓글']
}

yogi_df = pd.DataFrame(tmp)
yogi_df

,content,score,replyContent
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요,5,"이렇게 저희 매장 찾아주셔서 감사드립니다!!맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,5,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요고객님께서 맛있다고 말씀 주시니 피로가...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,5,수 많은 매장들을 헤치고 와주셔서 감사합니다!!저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,5,수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
...,...,...,...
18681,맛있게 잘 먹었어요,5,감사합니다!!! 다음에도 이용해주세용
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,4,죄송합니다 !! 다음에도 애용해주세용
18683,배달 빨라요 맛있게 잘 먹었습니당 !,5,감사합니다!!! 다음에도 이용해주세용
18684,잘 먹었어요,5,감사합니다!!! 다음에도 이용해주세용


In [115]:
# 데이터셋 합병 지점
# df = pd.concat([play_df[:3000], yogi_df[:3000]], axis=0, ignore_index=True)
df = pd.concat([yogi_df[:10000]], axis=0, ignore_index=True)
df

,content,score,replyContent
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요,5,"이렇게 저희 매장 찾아주셔서 감사드립니다!!맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,5,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요고객님께서 맛있다고 말씀 주시니 피로가...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,5,수 많은 매장들을 헤치고 와주셔서 감사합니다!!저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,5,수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
...,...,...,...
9995,먹을때마다 역시 맛있네요 김치찜은 여기서 맨날 먹습니다,5,감사합니다. 양도 더 늘리고 품질도 더 높이고 있습니다. 부족하시지 않게 맛있게...
9996,맛있게 잘먹었습니다,5,토핑폭탄을 방문해주셔서 감사합니다. 자주 이용해주셔서 너무 감사드려요. 더 많이 ...
9997,배달 25분만에왔어여 존맛이네요진짜고기도엄청많고 국물예술이에여!!!꼭시켜드세요 요청...,5,토핑폭탄을 이용해 주셔서 너무 감사드립니다. 바쁜시간에도 저희가게에 리뷰선물까지 주...
9998,새벽에 배고파서 허겁지겁 먹느라 대충 찍었네요 맛있었어요,5,토핑폭탄을 이용해 주셔서 너무 감사드립니다. 바쁜시간에도 저희가게에 리뷰선물까지 주...


In [116]:
# 불필요한 문자 제거

# 고객리뷰 부분
content_list = []
for content in df['content']:
    if isinstance(content, str):
        c_result = re.sub(r'\.{2,}', '.', content)
        c_result = re.sub(r'\,{2,}', ',', c_result)
        c_result = re.sub(r'\~{2,}', '~', c_result)
        c_result = re.sub(r'\!{2,}', '!', c_result)
        c_result = re.sub(r'\?{2,}', '?', c_result)
        c_result = re.sub(' +', ' ', c_result)

        content_list.append(c_result)
    else:
        content_list.append('')

df['content'] = content_list

# 사장댓글 부분
replyContent_list = []
for replyContent in df['replyContent']:
    if isinstance(replyContent, str):
        r_result = re.sub(r'\.{2,}', '.', replyContent)
        r_result = re.sub(r'\,{2,}', ',', r_result)
        r_result = re.sub(r'\~{2,}', '~', r_result)
        r_result = re.sub(r'\!{2,}', '!', r_result)
        r_result = re.sub(r'\?{2,}', '?', r_result)
        r_result = re.sub(' +', ' ', r_result)

        replyContent_list.append(r_result)
    else:
        replyContent_list.append('')

df['replyContent'] = replyContent_list

df.head()

,content,score,replyContent
0,사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요,5,"이렇게 저희 매장 찾아주셔서 감사드립니다!맛 뿐만 아니라 양도, 서비스도 모두 최상..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,5,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요고객님께서 맛있다고 말씀 주시니 피로가...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,5,수 많은 매장들을 헤치고 와주셔서 감사합니다!저희 매장의 메뉴를 제대로 즐기신 거 ...
4,맛있어요 양도많구좋음,5,수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...


In [117]:
for i in range(10):
    print(df['replyContent'].loc[i])
    print('------')

이렇게 저희 매장 찾아주셔서 감사드립니다!맛 뿐만 아니라 양도, 서비스도 모두 최상의 품질을 드리려고 노력하고 있습니다 모든 직원들이 힘내서 고객님의 마음을 잡기 위해 계속 노력하겠습니다항상 건강 챙기시고, 오늘도 즐거운 하루 보내시길 바라겠습니다 
------
좋은 리뷰 덕분에 기분이 행복해지는 것 같아요고객님께서 맛있다고 말씀 주시니 피로가 싹 사라지네요! 주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다시 시작할 수 있는 거 같습니다 정말 감사드려요오늘도 화이팅 있게 마무리하시길 바라요
------
소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 100 만족감을 드리고 싶었는데 그러지 못한 것 같아 너무 죄송합니다 말씀 주신 부분에 대해서는 주의하여 다음 주문시에는 실수 없도록 꼼꼼하게 확인하겠습니다.! 저희 매장을 찾아 주셔서 다시 한번 감사의 말씀드리며, 언제나 행복한 순간들만 가득하시길 바라겠습니다!
------
수 많은 매장들을 헤치고 와주셔서 감사합니다!저희 매장의 메뉴를 제대로 즐기신 거 같아 너무 영광이에요 모두 마음에 드셨을까요?전부 정성껏 준비했지만, 늘 주문해 주신 분들의 평가를 기다릴 땐 떨려요더욱 만족스러운 시간 드릴 수 있게 노력하겠습니다 
------
수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 성공이였을까요? 저희 메뉴 칭찬해 주셔서 감사합니다 고객님! 늘 어떻게하면 더 맛있게 드릴 수 있을까 고민 많이 하고 있어요항상 노력하고 발전하도록 하겠습니다항상 같은 만족감 드릴 수 있도록 언제나 최선을 다해 운영 하겠습니다 
------
저희 매장 찾아주신 것도 감사한데 리뷰까지.감동입니다고객님들의 배고픔을 저희는 지켜만 볼 수 없습니다! 언제든지 행복한 배부름을 느끼게 해 드릴 수 있도록 저희가 있지 않겠습니까!매번 찾아 주셔도 매번 시켜 먹고 싶은 매장이 되겠습니다 하시는 일 모두 번창하시길 진심으로 바라고 있을게요 
-----

In [118]:
shuffled_df = df.sample(frac=1, random_state=SEED)
shuffled_df

,content,score,replyContent
6252,오늘도 역시나 맛있게 잘 먹었습니다!,5,헤헤헤헿! 엄지척. 너무 좋아하는 엄지척. 앞으로도 더욱 열심히 만들어볼게요! 앞으...
4684,배달도 빠르고 맛있어요,5,별점 5개와 리뷰를 잊지 않고 남겨주셔서 감사합니다직접 드시는 건데 맛은 당연히 좋...
1731,배달도 빠르고 치킨도 너무 맛있게 잘 먹었습니다,5,저희 청년치킨을 이용해주셔서 감사드립니다맛있는 치킨과 푸짐한 양 신속한 서비스로 보...
4742,푸짐하고 맛있어서 한번씩 생각나요. 잘 먹었어요.,5,감사합니다 푸짐한 사진도 너무 감사해요 다양한 구성에 푸짐함까지 더해져 사랑받는 도...
4521,굴 너무 싱싱하고 맛있네요,5,안녕하세요'' 소중한 고객님!우리 고객님의 맛있다는 말은 언제들어도 기분이 좋네요언...
...,...,...,...
5734,콩나물국 맛있어요 잘 먹었습니다!,5,"고객님, 40여가지 재료로 당일 만든 정성이 가득한 음식입니다. 더욱 노력하겠습니다..."
5191,잘 먹었습니다!,5,명작파스타 입니다. 맛있게 드셨다니 저희도 기분이 좋습니다 정성스런 리뷰도 너무 감...
5390,종종 시켜먹는데 맛있어요!,5,저희 올데이파스타 동대문구점을 찾아주셔서 정말 대단히 감사합니다이렇게 맛있게 드셔주...
860,맛있게 잘 먹었습니다,5,안녕하세요 고돼지 송파점입니다별점만점 그리고 따뜻한 리뷰 작성해주셔서 감사합니다. ...


In [119]:
# from sklearn.model_selection import train_test_split

In [120]:
# train, test = train_test_split(df, test_size=0.3, random_state=SEED)
# train.shape, test.shape

In [121]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 0.18ubuntu0.18.04.1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [146]:
model_name = 'EasthShin/Youth_Chatbot_Kogpt2-base'

In [147]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [148]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [149]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

In [150]:
tokenizer.tokenize('오늘 저녁 뭐 먹을래?')

['▁오늘', '▁저녁', '▁뭐', '▁먹을', '래', '?']

In [151]:
tokenizer.eos_token_id

1

In [152]:
text = '오늘 저녁 뭐 먹을래?'
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids

tensor([[10070, 17969, 46651, 17003,  7383,   406]])

In [153]:
result_ids = model.generate(input_ids,
                            max_length=128, # 생성 최대 길이
                            repetition_penalty=2.0, # 반복 토큰 생성에 대한 패널티, 1보다 큰 수를 줘야함
                            use_cache=True, # 빠른 추론을 위한 캐시 여부
                            do_sample=True, # 확률적 샘플링
                            temperature=0.2, # 소프트맥스 온도
                            top_k=10
                            )
result_ids

tensor([[10070, 17969, 46651, 17003,  7383,   406, 39514,  8022, 22375,  9078,
          7892, 26714,  7890,  6824,  8084,   376,     1]])

In [154]:
result = tokenizer.decode(result_ids[0])
print(result)

오늘 저녁 뭐 먹을래?저녁에 무엇을 하실 계획이신가요!</s>


In [155]:
tokenizer([text, result], return_tensors='pt', padding=True)

{'input_ids': tensor([[10070, 17969, 46651, 17003,  7383,   406,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3],
        [10070, 17969, 46651, 17003,  7383,   406, 39514,  8022, 22375,  9078,
          7892, 26714,  7890,  6824,  8084,   376,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [156]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.content = shuffled_df['content'].tolist()
        self.replyContent = shuffled_df['replyContent'].tolist()
        
    def __len__(self):
        return len(self.content)

    def __getitem__(self, idx):
        return '<q>' + self.content[idx] + '</s><a>' + self.replyContent[idx] + '</s>'

In [157]:
def collate_fn(batch):
    x = tokenizer(batch, return_tensors='pt', padding=True)
    return {'x': x}

In [158]:
dt = ChatDataset(shuffled_df)
dl = torch.utils.data.DataLoader(dt, batch_size=2, collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   455,   405, 30899,  7235, 10330,  7055, 11355, 18479,  9443,
           9784,  8017, 16913,  7182,   376,     1,  9724,   439,   405,  8729,
           8729,  8729,     5,   376, 30762,  8360,   389, 12371, 22600, 30762,
           8360,   389, 12634,  7235,  9947, 22805,  9659,  7656,  6866,  8084,
            376, 12634,  7235,  9301,  9599,  9265,  8806,  9134, 18882, 29576,
           7489, 28709,  8084,  9166,  7894,  9267,  9098,  9947, 29918, 11355,
           9021, 28711,  7281,  6872, 16913,  7182, 11474,  7062,  8263, 10091,
          10126, 19272,  9443, 48064,  9075, 10061, 15730,  8702,  7801,  8084,
           8185,  8806,  9301,  8159, 39534, 11001,  9301, 31549,  8109, 34222,
          10873, 10762,  8146,  9481, 16913,  7182,   376, 11336, 10286, 22411,
          10554, 35723, 12298,  7177,  9036, 25474,  7671, 29207,  6992, 15844,
           6872, 16913,  7182,   376,  8330,   387,  7478,  7685, 10403,   428,
            411,   42

In [159]:
model(**batch['x']).logits.shape

torch.Size([2, 164, 51200])

In [160]:
tokenizer.pad_token_id

3

In [161]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()

    for batch in tqdm(dataloader):
        x = batch['x'].to(device)
        pred = model(**x).logits
        n_class = pred.shape[-1]
        pred = pred[:,:-1]
        pred = pred.reshape(-1, n_class)

        tgt = x['input_ids'][:,1:]
        tgt = tgt.flatten()

        mask = tgt != 3
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [162]:
batch_size = 4
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 20

In [163]:
import gc

In [164]:
reset_seeds(SEED)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

train_dt = ChatDataset(shuffled_df)
train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(train_loss)

    model.save_pretrained(f'{DATA_PATH}models/EasthShin/p0y10000e20_shuffle_ver.2_{i}')


  0%|          | 0/2500 [00:00<?, ?it/s]

1.7662056977808476


  0%|          | 0/2500 [00:00<?, ?it/s]

1.267013477599621


  0%|          | 0/2500 [00:00<?, ?it/s]

...

0.22277454029619695


  0%|          | 0/2500 [00:00<?, ?it/s]

0.21170884005725385


  0%|          | 0/2500 [00:00<?, ?it/s]

0.20308448314368724


In [168]:
@torch.no_grad()
def chatbot(model, tokenizer, max_len, device):
    model.eval()
    while True:
        text = input('user >> ').strip()
        if text == 'qwer':
            break

        text = '<q>' + text + '</s><a>'
        x = tokenizer.encode(text, return_tensors='pt').to(device)
        result_ids = model.generate(x,
                            max_length=128, # 생성 최대 길이
                            repetition_penalty=2.0, # 반복 토큰 생성에 대한 패널티, 1보다 큰 수를 줘야함
                            use_cache=True, # 빠른 추론을 위한 캐시 여부
                            do_sample=True, # 확률적 샘플링
                            temperature=0.8, # 소프트맥스 온도
                            top_k=10
                            )
        q_len = len(text) +1
        text = tokenizer.decode(result_ids[0])

        print('bot >> ', text[q_len:-4])

In [169]:
model = model.from_pretrained(f'{DATA_PATH}models/EasthShin/p0y10000e20_shuffle_ver.2_19').to(device)

In [170]:
chatbot(model, tokenizer, 256, device)

user >>  맛은 있었는데 양이 좀 적었어요.


bot >>  잭아저씨 족발보쌈을 찾아주셔서 감사합니다바쁘실텐데 잊지 않고 리뷰 남겨 주시니 넘넘 행복하네요 양에 맞춰서, 드시느라 힘들지만 맛은 항상 최고인 거 같아요,!앞으로도 만족만 하겠다는 말씀 담아 만들겠습니다이렇게 칭찬 가득한 하루를 보내시는 고객님들 덕분에 오늘도 힘을 얻고, 뿌듯함에 보답 드릴 수 있어 다시 한번 감사의 마음드립니다고객님의 소중한 시간에 저희가 함께 할 것을 약속하겠니, 자주 뵙고


user >>  배달이 많이 늦었는데 맛은 있었어요!


bot >>  wg님, 안녕하세요.일단 죄송하다는 말씀 먼저 드립니다 저희를 믿고 와주신 귀한 발걸음에 큰 불편함을 느끼셨을 것을 합니다. 기사님의 정확한 배차 지연에 대한 부분도 확인해 보니, 동일한 이슈가 발생하지 않도록 매장에서 꼼꼼하게 확인하고 전달 드릴 수 있도록 하겠습니다. 문제가 있으시다면 언제든 피드백 부탁드릴게요,저희가 고객님을 최우선으로 생각하며 화이팅 하는 모습을 보이겠습니다고객님이 항상 행복하시길 바라며 늘 건강 조심 하십시


user >>  맛있어요.


...

user >>  오랜만에 시켰는데 맛이 변했네요


bot >>  주문해 주셔서 감사합니다 맛있게 드셨다니 기분이 좋아지내요.고객님 리뷰 덕에.힘이 납니다.열심히 하겠습니다.감사드려요, 고객님이 편안하시는 내일도 행복하시길 바랄게용! 감기조심 하세요소중한 시간 내어 저희 매장 찾아해주신점 진심으로 감사의 말씀 드리겠습니다고객님의 신뢰에 오늘 하루가 따뜻함과 부드러움이 함께하기를 소망하겠습니다와아, 다음에 또 뵈어요.


user >>  오랜만에 시켰는데 맛이 변했네요.


bot >>  안녕하세요! 주문해주셔서 감사합니다''한번도 안드렸다가 한 번만 드신분은 없답니다.맛있게 잘 먹었다니 너무 감동이다앙항상 맛있을수 있도록 노력하겠습니당찜 꾸욱 눌러주시공청결 세스코에 가입되어 청결을 기하게 되었어영자주 찾아 주세용 꿔바로우에 분모자튀김은 선택이 아니라 필수에욤 저희는 매번 신선한 재료만을 사용하며 찜 서비스를 제공옵니다고객님들


user >>  qwer
